In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import tensorflow as tf
from keras.utils import to_categorical
import os
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [5]:
def Split_Train_Dev_Test(Num_batches , seed):

    np.random.seed(seed)

    p = np.random.permutation( Num_batches )
    p = p + 1
    p = p.tolist()
    
    # 80 10 10 split
    num_dev =  Num_batches//10
    num_test = Num_batches//10
    
    batch_train = p[0:-(num_dev+num_test)]
    batch_dev = p[-(num_dev+num_test):-num_test]
    batch_test = p[-num_test:]


    return batch_train, batch_dev, batch_test

In [6]:
def ImportValues(folder,batch,exmp_num):
   # This function loads the x and y values, reshapes X to volume and tranform Y to 1-hot 

    name_img = folder + 'X_Img_Values' + str(batch) + '.npy'
    name_sex = folder + 'X_Sex_Values' + str(batch) + '.npy'
    name_age = folder + 'X_Age_Values' + str(batch) + '.npy'
    namey = folder + 'YValues' + str(batch) + '.npy'
    
    X_img=np.load(name_img)
    X_sex=np.load(name_sex)
    X_age=np.load(name_age)
    Y=np.load(namey)
    Y=np.squeeze(Y)

    # length check
    assert(len(X_img) == len(Y))


    n_classes = 2 # 3 labels: 0:normal, 1:MCI, 2:Alzh

    X_img = np.reshape(X_img,(-1,116,130,83,1)) ######################

#     # choosing a 16 by 16 by 16 slot in the images
    # X_img = X_img[:, 100:108, 100:108, 100:108, :]
    
    # change lables to 1-hot form matrix
    Y = to_categorical(Y, n_classes)

    
    X_img = X_img[[exmp_num-1], :, :, :, :]
    X_sex = X_sex[[exmp_num-1], :]
    X_age = X_age[[exmp_num-1], :]
    Y = Y[[exmp_num-1], :]
    
    return X_img, X_age, X_sex ,Y


In [12]:


folder='../../.././Data_Set/XY_Values_BatchSize8_Scaled_TwoGroups (1)/'
save_name = '.././B8_3SimplArch_Aug_0.15AND0.20Drop_2Groups_NoSxAge_L2Regular0.5_biasReg0.5_convReg0.5/rep/CNN_trained_best'

#dimensions of our input and output
n_x = 116
n_y = 130
n_z = 83
n_classes = 2

#Choice of specific exmample
batch = [38]
exmp_num = 1

#Choice of sections and box size
size_box = 50
slice_x = 50
slice_y = 65
slice_z = 42


with tf.Session() as sess:
    
    #Load the model
  
    new_saver = tf.train.import_meta_graph(save_name + '.meta')
    new_saver.restore(sess, save_name)

    x_img_input = sess.graph.get_tensor_by_name('Input_img:0' )  
    x_age_input = sess.graph.get_tensor_by_name('Input_age:0' )
    x_sex_input = sess.graph.get_tensor_by_name('Input_sex:0' )
    y_input = sess.graph.get_tensor_by_name('Output:0') 
    cost = sess.graph.get_tensor_by_name('Cost:0' )
    
    prediction = sess.graph.get_tensor_by_name('Prediction/BiasAdd:0')
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = sess.graph.get_tensor_by_name('accuracy:0')   

    
    import datetime

    start_time = datetime.datetime.now()

    mini_batch_x_img , mini_batch_x_age , mini_batch_x_sex , mini_batch_y = ImportValues(folder, batch[0],exmp_num)
    
    param2 , intensity_orig , indx = sess.run([accuracy, tf.nn.softmax(prediction), tf.argmax(y_input, 1)], feed_dict={ x_img_input: mini_batch_x_img,
                            x_age_input: mini_batch_x_age, x_sex_input: mini_batch_x_sex, y_input: mini_batch_y})

    print(intensity_orig,indx)
    
    mini_batch_x_img_train = mini_batch_x_img*1
    
    #Initialize the heat map
    Heat_map_x = np.zeros((n_y//size_box,n_z//size_box))
    Heat_map_y = np.zeros((n_x//size_box,n_z//size_box))
    Heat_map_z = np.zeros((n_x//size_box,n_y//size_box))
                          
    np.save('intensity_orig.npy',intensity_orig)
    
    # create the data for heat map in x direction
    for i in range(n_y//size_box):
        for j in range(n_z//size_box):

            mini_batch_x_img_train = mini_batch_x_img*1
            mini_batch_x_img_train[0,slice_x-size_box//2:slice_x+size_box//2,i*size_box:(i+1)*size_box ,j*size_box:(j+1)*size_box,0] = 0

            print(mini_batch_x_img.shape , mini_batch_x_age.shape , mini_batch_x_sex.shape , mini_batch_y.shape)
            param2 , intensity , y_true = sess.run([accuracy, tf.nn.softmax(prediction), y_input], feed_dict={ x_img_input: mini_batch_x_img_train,
                                        x_age_input: mini_batch_x_age, x_sex_input: mini_batch_x_sex, y_input: mini_batch_y})
            
            Heat_map_x[i,j] = intensity[0][indx]       
  
    np.save('Heat_map_x.npy',Heat_map_x)
    
    # create the data for heat map in y direction    
    for i in range(n_x//size_box):
        for j in range(n_z//size_box):          

            mini_batch_x_img_train = mini_batch_x_img*1
            mini_batch_x_img_train[0,i*size_box:(i+1)*size_box ,slice_y-size_box//2:slice_y+size_box//2,j*size_box:(j+1)*size_box,0] = 0

            print(mini_batch_x_img.shape , mini_batch_x_age.shape , mini_batch_x_sex.shape , mini_batch_y.shape)
            param2 , intensity , y_true = sess.run([accuracy, tf.nn.softmax(prediction), y_input], feed_dict={ x_img_input: mini_batch_x_img_train,
                                        x_age_input: mini_batch_x_age, x_sex_input: mini_batch_x_sex, y_input: mini_batch_y})
            
            Heat_map_y[i,j] = intensity[0][indx]       
  
    np.save('Heat_map_y.npy',Heat_map_y)
    
    # create the data for heat map in z direction    
    for i in range(n_x//size_box):
        for j in range(n_y//size_box):

            mini_batch_x_img_train = mini_batch_x_img*1
            mini_batch_x_img_train[0,i*size_box:(i+1)*size_box ,j*size_box:(j+1)*size_box,slice_z-size_box//2:slice_z+size_box//2,0] = 0

            print(mini_batch_x_img.shape , mini_batch_x_age.shape , mini_batch_x_sex.shape , mini_batch_y.shape)
            param2 , intensity , y_true = sess.run([accuracy, tf.nn.softmax(prediction), y_input], feed_dict={ x_img_input: mini_batch_x_img_train,
                                        x_age_input: mini_batch_x_age, x_sex_input: mini_batch_x_sex, y_input: mini_batch_y})
            
            Heat_map_z[i,j] = intensity[0][indx]       
  
    np.save('Heat_map_z.npy',Heat_map_z)
        
                                           
    end_time = datetime.datetime.now()
    print('Time elapse: ', str(end_time - start_time))





INFO:tensorflow:Restoring parameters from .././B8_3SimplArch_Aug_0.15AND0.20Drop_2Groups_NoSxAge_L2Regular0.5_biasReg0.5_convReg0.5/rep/CNN_trained_best
[[ 0.22844845  0.77155155]] [1]
(1, 116, 130, 83, 1) (1, 1) (1, 1) (1, 2)
(1, 116, 130, 83, 1) (1, 1) (1, 1) (1, 2)
(1, 116, 130, 83, 1) (1, 1) (1, 1) (1, 2)
(1, 116, 130, 83, 1) (1, 1) (1, 1) (1, 2)
(1, 116, 130, 83, 1) (1, 1) (1, 1) (1, 2)
(1, 116, 130, 83, 1) (1, 1) (1, 1) (1, 2)
Time elapse:  0:01:36.562088
